In [1]:
# Print out the orders

# The ID of the order form 
# See https://newmarketchurch.breezechms.com/forms/entries/557986
shopper_form_id = '557986'
# shopper_form_id = '6793f4'


In [2]:
# This is the file that was downloaded from 
order_form_entries = r"https://newmarketchurch.breezechms.com/forms/entries/{form_id}".format(form_id = shopper_form_id)
print("Download as Excel from {orders}, and paste path into weekly_orders below.".format(orders = order_form_entries))

Download as Excel from https://newmarketchurch.breezechms.com/forms/entries/557986, and paste path into weekly_orders below.


In [3]:
from datetime import datetime, timedelta

# With the API download of orders, it's no longer needed to download the file.
# This is the file that was downloaded from https://newmarketchurch.breezechms.com/forms/entries/557986
# weekly_orders = r"C:\Users\ralph\Downloads\PANTRY-ORDER-FORM-12723 (3).xlsx"

# title_date is the Thursday of this week.
current_run = datetime.now()
thursday = current_run + timedelta(days = 3 - current_run.weekday())
title_date = thursday.strftime('%m/%d/%Y')

# title_date = '12/14/2023'

last_run = '12/06/2023 12:00'
runmode = 'all'
# runmode = 'update'

# The number of labels to print for each shopper
number_of_labels = 6

In [4]:
# Get the exported filename for use in the report.
import os
from pathlib import Path

local_path = Path(os.environ['TEMP'])

filename = 'NCC Food Pantry Orders' 
title = filename + ' ' + title_date

# These files are created by this script.
order_print_file = os.path.join(os.path.dirname(local_path), filename +'.html')
label_file = os.path.join(os.path.dirname(local_path), filename +' labels.html')


In [5]:
# Choose the time period for the orders based on the runmode and current time.

modes = {
    'all': 'All orders received this week',
    'update': 'All orders received since {date}'.format(date = last_run),
    'special': 'Scpecial criteria were used.'
    }

# Only process recent orders.
if runmode == 'all':
    starttime = datetime.strptime(title_date, '%m/%d/%Y') - timedelta(days=5)
elif runmode == 'update':
    starttime = datetime.strptime(last_run, '%m/%d/%Y %H:%M')
else:
    starttime = datetime.today() - timedelta(days=7)


In [6]:
import numpy as np
import pandas as pd

# Read in the downloaded Excel file 
# allorders = pd.read_excel(weekly_orders)
# ordercount = len(allorders.index)
# print('{count} orders in input.'.format(count = ordercount))

In [7]:
# Find the event for this week's shopping event.
# Check in https://yoursubdomain.breezechms.com/api/events/attendance/add
# The Python wrapper suggested in https://app.breezechms.com/api seems to have major problems and no support.
# I'm trying https://pypi.org/project/breeze-chms-api/
from breeze_chms_api import breeze

# Initialize API 
breeze_api = breeze.breeze_api(breeze_url='https://newmarketchurch.breezechms.com',
                               api_key='8dfd0a0d7f5aaec745a73542f58eb8ba')

# Get all the people
# people = breeze_api.list_people()
# display(len(people))

# Find the shopping event by name and date.
# events = breeze_api.list_events(start=title_date, end=title_date)
# shoppingevent = [e for e in events if e['name'] == 'Food Pantry'][0]
# display(shoppingevent)

# Get the checked-in shoppers for the shoppingevent.
# shoppers = breeze_api.list_attendance(instance_id=shoppingevent['id'])
# display(shoppers)
# https://yoursubdomain.breezechms.com/api/events/attendance/list?instance_id=1521321&type=person

In [8]:
from collections import OrderedDict

# Build a DataFrame for all the orders. 
# This code builds the data to look like what's downloaded in an excel file from https://newmarketchurch.breezechms.com/forms/entries/557986.

# Get the order form entries.
online_orders = breeze_api.list_form_entries(form_id = shopper_form_id, details=True)
# The entry response array has key values that correspond to the form fields.
ordercount = len(online_orders)
print('{count} orders in input.'.format(count = ordercount))

# Get the form fields needed to make sense of the entries.
form_fields = breeze_api.list_form_fields(form_id = shopper_form_id)

# "Join" the order entries with the form fields.

shopper_ids = []
all_api_orders = []

for order in online_orders:
    row = OrderedDict()
    for field, value in order['response'].items():
        field = [f for f in form_fields if f['field_id'] == field][0]
        # print(field, '\nv---', type(value), value)
        # 'Name' and 'Address' values are dicts specific to those data types.
        if field['field_type'] == 'name':
            row['Date'] = value['created_on']
            row['First Name'] = value['first_name']
            row['Last Name'] = value['last_name']
        elif field['field_type'] == 'address':
            value = [v for v in value if v['is_primary'] == '1'][0]
            row['Street Address'] = value['street_address']
            row['City'] = value['city']
            row['State'] = value['state']
            row['Zip'] = value['zip']
        else:
            # If the value is a dict, look it up in the form fields.
            # If it is a list of dicts, look up each and concatenate them with '<br >'.
            if isinstance(value, dict):
                value = [op for op in field['options'] if op['option_id'] == value['value']][0]['name']
            if isinstance(value, list):
                selections = []
                for onevalue in value:
                    lookup = [op for op in field['options'] if op['option_id'] == onevalue['value']]
                    if len(lookup) == 0:
                        print("Order from {fname} {lname} on {date} includes item not on the form.".format(order=order['response'], fname =row['First Name'], lname = row['Last Name'], date=row['Date']))
                    else: 
                        selections.append(lookup[0]['name'])
                value = '<br />'.join(selections)
            
            row[field['name']] = value
    if pd.to_datetime(row['Date']) >= starttime:
        shopper_ids.append(order['person_id'])   
        all_api_orders.append(row)

allorders = pd.DataFrame(all_api_orders)
printed = len(allorders.index)
print('{count} orders filtered by date and time.'.format(count = printed))

54 orders in input.
Order from Judy Greenfield on 2023-12-10 23:04:00 includes item not on the form.
Order from Ernest Beaulieu on 2023-12-06 06:33:54 includes item not on the form.
Order from Farren Lydston on 2023-12-05 21:12:30 includes item not on the form.
Order from Katherine  Salisbury  on 2023-12-05 19:05:09 includes item not on the form.
Order from lynne rocco on 2023-12-05 16:48:56 includes item not on the form.
Order from Marci  Linscott on 2023-12-05 10:43:55 includes item not on the form.
Order from Darlene Colvin on 2023-12-05 09:37:31 includes item not on the form.
Order from Darlene Colvin on 2023-12-05 09:37:31 includes item not on the form.
Order from Dana Glennon on 2023-12-05 06:13:00 includes item not on the form.
33 orders filtered by date and time.


In [9]:
# Coalesce columns to simplify the order form.

# Combine first and last name.
allorders.insert(1, 'Name', allorders['First Name'] + ' ' + allorders['Last Name'])

# Format the Zip code correctly.
allorders['Zip'] = allorders['Zip'].astype(str).str.pad(5,fillchar='0')

# Combine address elements into a single address field.
allorders.insert(5, 'Address', allorders['Street Address'].astype(str) + '<br/>' + 
                                 allorders['City'] + ' ' + allorders['State'] + ' ' + allorders['Zip'])

# Replace "NaN" values with blanks.
allorders = allorders.replace(np.nan, '')

# Drop the fields we combined.
allorders = allorders.drop(columns=['Date', 'First Name', 'Last Name', 'Street Address', 'City', 'State', 'Zip'])

In [10]:
# This cell was run once to save the orders for recurring orders.
# Edit as appropriate and rerun to update the recurring orders.

# recurring = allorders[allorders['Name'] == 'Rosa Soto']
# recurring = recurring.drop(recurring.columns[1], axis=1)
# recurring.to_pickle('recurring.pk1')


In [11]:
# Add recurring orders - orders that have been saved and need to be filled even without a current order form.

# Get recurring orders that don't have order forms. 
recurringorders = pd.read_pickle('recurring.pk1')
recurringshoppers = ['30397406',   # Rosa Soto
                     '30397442']   # Jay Stillman

paper_order_labels = [
    {
        'shoppername': 'Jay Stillman', 
        'pickup': 'Thursday, 12 - 2pm', 
        'address': ' ', 
        'phone': '659-4911'
    }
    ]

if runmode == 'all':
    allorders = pd.concat([allorders, recurringorders])

In [12]:
# Check in the shoppers from the order forms for the shopping event.
# Note: All orders must be connected to People to allow check-in.

shopper_ids.extend(recurringshoppers)
print('{count} shoppers to check in.'.format(count = len(shopper_ids)))

# Find the shopping event by name and date.
events = breeze_api.list_events(start=title_date, end=title_date)
shoppingevent = [e for e in events if e['name'] == 'Food Pantry'][0]

for id in shopper_ids:
    check = breeze_api.event_check_in(person_id=id, instance_id=shoppingevent['id'])
    # display(check)

35 shoppers to check in.


In [13]:
# Aggregate multiple orders from the same name.

duporderers = set()

def concatenate(ser):
    # Collect all the requests from all the orders and include each one just once.
    br = '<br />'    # Order request seperator
    contents = set()
    if len(ser) > 1:
        # Keep track of the names on multiple orders so they can be included on the cover sheet. 
        duporderers.add(allorders['Name'].loc[ser.index[0]])
    for elem in ser:
        contents = contents.union(set((str(elem).split(br))))
    contents.discard('')   # Don't keep blank requests.
    return br.join(list(contents))
        
allorders = allorders.groupby('Name', as_index=False).aggregate(concatenate)

allorders = allorders.sort_values(by = 'Pickup Time')

deduped = len(allorders.index)
print('{count} orders deduped.'.format(count = deduped))
if len(duporderers) > 0:
    print('Duplicate orders receivd from {people}'.format(people = duporderers))

34 orders deduped.


In [14]:
# Collect the summary for only refrigerated items. ("page 1")
# Include only the following fields in the Summary.
summary = allorders[[
    'Name', 
    'Number of people in household', 
    'Pickup Time',
    'Address', 
    'Email', 
    'Phone', 
    'MEATS/FROZEN ITEMS', 
    'REFRIGERATED ITEMS', 
]]

In [15]:
# Define HTML templates using Jinja2 for printing the data.

from jinja2 import Template

# Template for the whole report, including style.

orders = Template('''
<!DOCTYPE html>
<style>
    h1 { 
            font-family: Arial;
        }
    table tr td { font-family: Arial; font-size: 4mm; }
    table { width: 100%; }
    th { font-family: Arial; font-size: 6mm; }
    td { border-bottom: 1px solid #ddd; }
    td.category { width: 35%; }
</style>
<html lang="en">
<head>
<meta charset="utf-8"/>
<meta name="viewport" content="width=device-width"/>
<title>{{title}}</title>
</head>
<body>
{{body}}
</body>
</html>
''')

# Template for each shopper
shopperhtml = Template('''
<h1 style="page-break-before:right;" class="shopper-name">PANTRY ORDER FORM ({{date}})</h1>
<table>
<thead><tr><th colspan=2>{{shoppername}}</th></tr></thead>
{{data}}
</table>
''')

coversheet = Template('''
<h2>NCC Pantry Order Print Cover Sheet for {{date}}</h2>
<dl>
<dt>Total number of orders in this print file</dt>
<dd><b>{{deduped}}</b></dd>
<dt>Order forms in input file (before removing old and duplicate orders)</dt>
<dd>{{received}}</dd>
<dt>"Recurring" orders included without an online order form</dt>
<dd>{{recurring}}</dd>
<dt>Printing orders since</dt>
<dd>{{starttime}} - {{modemsg}}</dd>
<dt>Report run at</dt>
<dd>{{runtime}}</dd>
<dt>Input File</dt>
<dd>{{input_file}}</dd>
<dt>Output File</dt>
<dd>{{output_file}}</dd>
{% for duper in dupers %}
    <dt>Multiple Orders received from</dt>
    <dd>{{duper}}</dd>
{% endfor %}
</dl>
''')

sectionheader = '''
<h1 style="page-break-before:always;">One Page Summaries for Refrigerated Items</h1>
'''

# Template for each row
rowhtml = Template('<tr><td class="category">{{key}}</td><td>{{val}}</td></tr>')


In [16]:
# Format the order forms as html.

def formatshoppers(date, data):
    output = ''
    
    for _, row in data.iterrows():
        rowtext = ''
        # Format each field as a row in a table.
        for i in range(len(data.columns)):
            rowtext += rowhtml.render({'key' : data.columns[i],
                                       'val' : row.iloc[i]})
        
        # Create a page using the above table.
        output += shopperhtml.render({'date': date, 'data': rowtext, 'shoppername': row['Name']})
    
    return output

In [17]:
# Build the report using Jinja2.

# Cover Sheet
output = coversheet.render({'received': ordercount, 
                            'printed': printed, 
                            'deduped': deduped, 
                            'recurring': len(recurringorders),
                            'date': title_date,
                            'input_file': 'from API',
                            'output_file': order_print_file, 
                            'dupers': duporderers,
                            'starttime': starttime,
                            'modemsg': modes[runmode],
                            'runtime': current_run.strftime("%Y-%m-%d %H:%M:%S"),
                           })

# Full Orders
output += formatshoppers(title_date, allorders) 

# Separator
output += sectionheader

# Refrigerated items
output += formatshoppers(title_date, summary)

orders_html = orders.render({'body': output})

In [18]:
# Define HTML templates using Jinja2 for printing labels

from jinja2 import Template

# Template for the address labels, including style.
# Styled to fit Avery 8163 (2" x 4") labels. https://www.avery.com/help/article/avery-labels-2-inch-x-4-inch

labels = Template('''<!DOCTYPE html>
<style>
    @page {
        margin-top: 0.45in;
        margin-bottom: 0.45in;
        margin-left: 0.0in;
        margin-right: 0.00in;    
        }
.label {
  width: 3.40in;
  height: 1.875in;
  padding-top: 0.0in;
  padding-bottom: 0.125in;
  padding-left: 0.25in;
  padding-right: 0.25in;
  border-width: 0.0in;
  margin-left: 0.125in;
  margin-right: 0.125in;
  margin-top: 0.0in;
  margin-bottom: 0.0in;
  float: left;
  font-family: Arial;
  font-size: 0.9em;
  text-align: left;
  overflow: hidden;
  outline: 0px white;
  page-break-inside: avoid;
}
.name {
  margin-top: 0.125in;
  font-size: 1.5em;
}
</style>
<html lang="en">
<head>
<meta charset="utf-8"/>
<meta name="viewport" content="width=device-width"/>
</head>
<body>{{body}}</body>
</html>''')

# Template for each shopper
shopperlabel = Template('''
<div class="label">
<p class='name'>{{shoppername}}</p>
<p>{{pickup}}</p>
<p>{{address}}</p>
<p>{{phone}}</p>
</div>
''')



In [19]:
# Create labels to be attached to the bags for the orders.

output = ''
# Manually add people for paper orders.
for po in paper_order_labels:
    output += number_of_labels * shopperlabel.render(po)

for _, row in summary.iterrows():
    output += number_of_labels * shopperlabel.render({
        'shoppername': row['Name'], 
        'pickup': row['Pickup Time'], 
        'address': row['Address'], 
        'phone': row['Phone']
    })

labels_html = labels.render({'body': output})

In [20]:
# Optional: Display the report here.
# import IPython
# IPython.display.HTML(orders_html)
# display(orders_html)

In [21]:
# Use Rapid API yakpdf - HTML to PDF to format the html output as pdf for printing.
import requests

def to_pdf(source_html):
    # Using https://rapidapi.com/yakpdf-yakpdf/api/yakpdf with limited free license.

    url = "https://yakpdf.p.rapidapi.com/pdf"

    payload = {
    	"source": { "html": source_html },
    	"pdf": {
    		"format": "Letter",
    		"scale": 1,
    		"printBackground": False
    	},
    	"wait": {
    		"for": "navigation",
    		"waitUntil": "load",
    		"timeout": 2500
    	}
    }
    headers = {
    	"content-type": "application/json",
    	"X-RapidAPI-Key": "bcf6330bd6msh0670320c9453831p16412djsn374499f34ed2",
    	"X-RapidAPI-Host": "yakpdf.p.rapidapi.com"
    }

    response = requests.post(url, json=payload, headers=headers)

    return response.content


In [24]:
# Write out the files to print.

order_pdf_file = os.path.splitext(order_print_file)[0]+'.pdf'
# with open(order_pdf_file, 'wb') as f:
#     f.write(to_pdf(orders_html))
# print('Wrote {file}'.format(file = order_pdf_file))

label_pdf_file = os.path.splitext(label_file)[0]+'.pdf' 
# with open(label_pdf_file, 'wb') as f:
#     f.write(to_pdf(labels_html))
# print('Wrote {file}'.format(file = label_pdf_file))

In [27]:
# Import smtplib for the actual sending function.
import smtplib

# Perhaps host my own SMTP server: https://medium.com/@coffmans/setup-your-own-simple-smtp-server-how-to-c9159cfc7934

# Here are the email package modules we'll need.
from email.message import EmailMessage

SMTPserver = 'gmail.com'
sender =     'ralphcase@gmail.com'
destination = 'ralphcase@hotmail.com'

USERNAME = "ralphcase"
PASSWORD = "ell3nJ34n3tteG3n3s1s"

# Create the container email message.
msg = EmailMessage()
msg['Subject'] = 'Print files for {title}'.format(title = title)
# me == the sender's email address
# family = the list of all recipients' email addresses
msg['From'] = sender
msg['To'] = destination
msg.preamble = 'You will not see this in a MIME-aware mail reader.\n'
msg.set_content('Please print the 2 pdf attachments.')


# Open the files in binary mode.  You can also omit the subtype
# if you want MIMEImage to guess it.
files_to_send = [order_pdf_file, label_pdf_file]
for file in files_to_send:
    with open(file, 'rb') as attachment:
        msg.add_attachment(attachment.read(), maintype='application',
                                 subtype='pdf', 
                                 # subtype='octet-stream', 
                          filename = attachment.name)

# Send the email via Gmail
server = smtplib.SMTP_SSL('smtp.gmail.com', 465)
server.login(sender, PASSWORD)
server.send_message(msg)
server.quit()


SMTPAuthenticationError: (534, b'5.7.9 Application-specific password required. For more information, go to\n5.7.9  https://support.google.com/mail/?p=InvalidSecondFactor i15-20020ac871cf000000b004257afd873fsm6715940qtp.35 - gsmtp')